In [3]:
import pandas as pd

ratings=pd.read_csv('u.data',sep='\t',names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings.drop('timestamp', axis=1, inplace=True)
print(ratings.head)

<bound method NDFrame.head of        user_id  movie_id  rating
0          196       242       3
1          186       302       3
2           22       377       1
3          244        51       2
4          166       346       1
...        ...       ...     ...
99995      880       476       3
99996      716       204       5
99997      276      1090       1
99998       13       225       2
99999       12       203       3

[100000 rows x 3 columns]>


In [4]:
user_item=ratings.pivot_table(index='user_id',columns='movie_id',values='rating')
print(user_item.head())

movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2          4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5          4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5          NaN   NaN   NaN   NaN  

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

user_filled=user_item.fillna(0)

item_matrix=user_item.fillna(0)  #item
item_matrix_T = item_matrix.T    #item
item_similarity = cosine_similarity(item_matrix_T) #item
item_similarity_df = pd.DataFrame(item_similarity, index=item_matrix_T.index, columns=item_matrix_T.index)


user_similarity=cosine_similarity(user_filled)
df=pd.DataFrame(user_similarity,index=user_item.index,columns=user_item.index)
print(df.head())
df.loc[2].sort_values()

user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.166931  0.047460  0.064358  0.378475  0.430239  0.440367   
2        0.166931  1.000000  0.110591  0.178121  0.072979  0.245843  0.107328   
3        0.047460  0.110591  1.000000  0.344151  0.021245  0.072415  0.066137   
4        0.064358  0.178121  0.344151  1.000000  0.031804  0.068044  0.091230   
5        0.378475  0.072979  0.021245  0.031804  1.000000  0.237286  0.373600   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.319072  0.078138  0.376544  ...  0.369527  0.119482  0.274876   
2        0.103344  0.161048  0.159862  ...  0.156986  0.307942  0.358789   
3        0.083060  0.061040  0.065151  ...  0.031875  0.042753  0.163829   
4        0.188060  0.101284  0.060859  ...  0.052107

,2
user_id,
571,0.000000
670,0.000000
172,0.000000
778,0.007685
565,0.012141
...,...
131,0.476880
460,0.504418
931,0.512473


In [23]:
def show_movie_titles(movie_ids, movies_df):
    df = pd.DataFrame({'movie_id': movie_ids})
    df = df.merge(movies_df, on='movie_id', how='left')
    return df[['movie_id', 'movie_title']]

In [17]:
target_user=1
s_users=df.loc[target_user]
s_users=s_users.drop(index=target_user).sort_values(ascending=False)

print(s_users.head())

top_user=s_users.head(10).index
r_users=user_item.loc[top_user]
print(r_users.head())

user_ratings = user_item.loc[target_user].dropna()
liked_movies = user_ratings[user_ratings >= 4]

user_id
916    0.569066
864    0.547548
268    0.542077
92     0.540534
435    0.538665
Name: 1, dtype: float64
movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
916        4.0   3.0   3.0   4.0   3.0   NaN   4.0   NaN   5.0   NaN  ...   
864        5.0   4.0   NaN   4.0   4.0   NaN   5.0   5.0   5.0   NaN  ...   
268        3.0   2.0   1.0   4.0   NaN   NaN   4.0   NaN   NaN   4.0  ...   
92         4.0   3.0   NaN   4.0   4.0   NaN   4.0   5.0   4.0   NaN  ...   
435        5.0   4.0   3.0   4.0   2.0   NaN   4.0   3.0   4.0   5.0  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
916        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   3.0  
864        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
268        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

Movie Recommendations using item

In [18]:
scores = pd.Series(dtype='float64')

for movie_id, rating in liked_movies.items():
    similar_scores = item_similarity_df[movie_id] * rating
    scores = scores.add(similar_scores, fill_value=0)

Filter already seen movies

In [20]:
recommended = scores.drop(user_ratings.index, errors='ignore')

top_recommendations = recommended.sort_values(ascending=False).head(10)

print("Top Recommended Movie IDs:", top_recommendations.index.tolist())

Top Recommended Movie IDs: [423, 655, 318, 357, 568, 403, 385, 483, 433, 367]


Movie Titles

In [21]:
top_df = top_recommendations.reset_index()
top_df.columns = ['movie_id', 'score']
top_df = top_df.merge(movies, on='movie_id')
print(top_df[['movie_title', 'score']])

                              movie_title       score
0       E.T. the Extra-Terrestrial (1982)  301.454880
1                      Stand by Me (1986)  281.806850
2                 Schindler's List (1993)  278.354491
3  One Flew Over the Cuckoo's Nest (1975)  275.833487
4                            Speed (1994)  274.996916
5                           Batman (1989)  271.503827
6                        True Lies (1994)  269.864722
7                       Casablanca (1942)  269.032542
8                         Heathers (1989)  266.377917
9                         Clueless (1995)  266.160860


Avg rating for unseen movies

In [8]:
seen_movies=user_item.loc[target_user].dropna().index

unseen_ratings=r_users.drop(columns=seen_movies)

recommendations=unseen_ratings.mean().sort_values(ascending=False)

print("Top movie IDs to recommend to user 1:\n")
print(recommendations.head(10))

Top movie IDs to recommend to user 1:

movie_id
1168    5.0
313     5.0
1009    5.0
1019    5.0
1007    5.0
963     5.0
853     5.0
855     5.0
736     5.0
632     5.0
dtype: float64


In [9]:

movie_cols = ['movie_id', 'movie_title', 'release_date', 'video_release', 'IMDb', 'unknown', 'Action', 'Adventure',
              'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('u.item', sep='|', encoding='latin-1', names=movie_cols, usecols=[0, 1,2])

print(movies.head())


   movie_id        movie_title release_date
0         1   Toy Story (1995)  01-Jan-1995
1         2   GoldenEye (1995)  01-Jan-1995
2         3  Four Rooms (1995)  01-Jan-1995
3         4  Get Shorty (1995)  01-Jan-1995
4         5     Copycat (1995)  01-Jan-1995


In [10]:
recommendations_df=recommendations.reset_index()
recommendations_df.columns = ['movie_id', 'predicted_rating']

recommendations_df = recommendations_df.merge(movies, on='movie_id')
print(recommendations_df[['movie_title', 'predicted_rating']].head(10))

                                 movie_title  predicted_rating
0                       Little Buddha (1993)               5.0
1                             Titanic (1997)               5.0
2                     Stealing Beauty (1996)               5.0
3  Die xue shuang xiong (Killer, The) (1989)               5.0
4                 Waiting for Guffman (1996)               5.0
5    Some Folks Call It a Sling Blade (1993)               5.0
6                           Braindead (1992)               5.0
7                                Diva (1981)               5.0
8                         Shadowlands (1993)               5.0
9                     Sophie's Choice (1982)               5.0


PrecisionK evaluation for user

In [25]:
import numpy as np

# 1️⃣ Choose a user
target_user = 100

# 2️⃣ Get all movies they rated (ground truth)
user_ratings = user_item.loc[target_user].dropna()

# If user has rated fewer than K movies, choose another
if len(user_ratings) < 10:
    print("Choose a user with at least 10 ratings")
else:
    # 3️⃣ Randomly select K of them to hide
    K = 10
    hidden_movies = np.random.choice(user_ratings.index, size=K, replace=False)

    # 4️⃣ Temporarily remove them from the user's ratings
    user_item_hidden = user_item.copy()
    user_item_hidden.loc[target_user, hidden_movies] = np.nan

    # Re-run recommendation with updated matrix (fill NaNs)
    filled_matrix = user_item_hidden.fillna(0)
    user_sim = cosine_similarity(filled_matrix)
    sim_df = pd.DataFrame(user_sim, index=filled_matrix.index, columns=filled_matrix.index)

    similar_users = sim_df.loc[target_user].drop(index=target_user).sort_values(ascending=False)
    top_users = similar_users.head(40).index
    top_ratings = user_item.loc[top_users]

    unseen_movies = top_ratings.drop(columns=user_item.loc[target_user].dropna().index)
    pred_ratings = unseen_movies.mean().sort_values(ascending=False)

    # Top K recommendations
    top_k_recommendations = pred_ratings.head(K).index

    # 5️⃣ Evaluate: how many of the hidden ones are in top K?
    hits = set(hidden_movies).intersection(set(top_k_recommendations))
    precision_at_k = len(hits) / K

    print(f"Hidden movies: {hidden_movies}")
    print(f"Top-{K} recommendations: {top_k_recommendations.tolist()}")
    print("Top Recommended Movie Titles (User-Based):")
    print(show_movie_titles(top_recommendations.index, movies))

    print(f"Hits (correctly recommended): {hits}")
    print(f"Precision@{K}: {precision_at_k:.2f}")



Hidden movies: [ 333  690  886  751 1236  895  313  990  874  294]
Top-10 recommendations: [9, 1293, 1175, 276, 884, 86, 100, 1612, 513, 511]
Top Recommended Movie Titles (User-Based):
   movie_id                             movie_title
0       423       E.T. the Extra-Terrestrial (1982)
1       655                      Stand by Me (1986)
2       318                 Schindler's List (1993)
3       357  One Flew Over the Cuckoo's Nest (1975)
4       568                            Speed (1994)
5       403                           Batman (1989)
6       385                        True Lies (1994)
7       483                       Casablanca (1942)
8       433                         Heathers (1989)
9       367                         Clueless (1995)
Hits (correctly recommended): set()
Precision@10: 0.00


PrecisionK evaluation for item

In [26]:
import numpy as np

# Choose a target user with enough ratings
target_user = 1
user_ratings = user_item.loc[target_user].dropna()

# Filter to only movies with sufficient ratings
popular_rated = [movie for movie in user_ratings.index if user_item[movie].count() >= 30]

K = 5
if len(popular_rated) >= K:
    # Randomly hide K liked movies
    hidden_movies = np.random.choice(popular_rated, size=K, replace=False)

    # Create a modified rating matrix with hidden movies removed
    modified_ratings = user_ratings.drop(index=hidden_movies)
    liked_movies = modified_ratings[modified_ratings >= 4]

    # Score similar movies using item-based filtering
    scores = pd.Series(dtype='float64')
    for movie_id, rating in liked_movies.items():
        similar_scores = item_similarity_df[movie_id] * rating
        scores = scores.add(similar_scores, fill_value=0)

    # Drop already seen movies and sort
    recommendations = scores.drop(modified_ratings.index, errors='ignore')
    top_k_recommendations = recommendations.sort_values(ascending=False).head(K).index

    # Evaluate hits
    hits = set(hidden_movies).intersection(set(top_k_recommendations))
    precision_at_k = len(hits) / K

    print("Hidden movies:", hidden_movies)
    print("Titles of Hidden Movies (Actual Likes):")
    print(show_movie_titles(hidden_movies, movies))
    print("Top-K recommendations:", list(top_k_recommendations))
    print("Top Recommended Movie Titles (Item-Based):")
    print(show_movie_titles(top_k_recommendations, movies))

    print("Hits (correctly recommended):", hits)
    print(f"Precision@{K}:", round(precision_at_k, 2))
else:
    print("Not enough popular movies rated by the user to evaluate.")


Hidden movies: [229  69 219  73 139]
Titles of Hidden Movies (Actual Likes):
   movie_id                                 movie_title
0       229  Star Trek III: The Search for Spock (1984)
1        69                         Forrest Gump (1994)
2       219           Nightmare on Elm Street, A (1984)
3        73                             Maverick (1994)
4       139                        Love Bug, The (1969)
Top-K recommendations: [69, 423, 655, 318, 357]
Top Recommended Movie Titles (Item-Based):
   movie_id                             movie_title
0        69                     Forrest Gump (1994)
1       423       E.T. the Extra-Terrestrial (1982)
2       655                      Stand by Me (1986)
3       318                 Schindler's List (1993)
4       357  One Flew Over the Cuckoo's Nest (1975)
Hits (correctly recommended): {69}
Precision@5: 0.2
